# <div style="text-align: center">IIT Guwahati Summer Analytics Hackathon </div> 

The dataset is provided by DeltaX is the pioneering cross-channel digital advertising platform. The cloud-based platform leverages big data, user behavior, and machine learning algorithms to improve performance across the business funnel of advertisers.

## Problem Statement

Let's take a case where an advertiser on the platform (DeltaX) would like to estimate the performance of their campaign in the future.

Imagine it is the first day of March and you are given the past performance data of ads between 1st August to 28th Feb. You are now tasked to predict an ad's future performance (revenue) between March 1st and March 15th. Well, it is now time for you to put on your problem-solving hats and start playing with the data provided under the "data" section.

In [118]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [119]:
x_train = pd.read_csv(r'Train_Data.csv')
x_test = pd.read_csv(r'Test_Data.csv')

In [120]:
x_train = x_train.drop('date',axis=1)
x_train = x_train.drop('campaign',axis=1)
x_train = x_train.drop('adgroup',axis=1)
x_train = x_train.drop('ad',axis=1)
y_train = x_train['revenue']
x_train = x_train.drop('revenue',axis=1)
#separte the predicting attribute into Y for model training 

In [121]:
y_train

0         0.00
1         0.00
2         0.00
3         0.00
4       925.71
         ...  
4566      0.00
4567      0.00
4568     93.20
4569      0.00
4570      0.00
Name: revenue, Length: 4571, dtype: float64

In [122]:
x_train

,impressions,clicks,cost,conversions
0,24,6,0.08,0
1,1,0,0.00,0
2,13,4,0.04,0
3,5,4,0.08,0
4,247,126,1.29,4
...,...,...,...,...
4566,19,6,0.07,0
4567,1,0,0.00,0
4568,16,8,0.12,1
4569,37,13,0.23,0


In [123]:
cpr=x_train["clicks"]/x_train["impressions"]
cpc=x_train["cost"]/x_train["clicks"]

x_train.insert(2, "cpr", cpr)
x_train.insert(2, "cpc", cpc)
x_train['cpc'].fillna((x_train['cpc'].mean()), inplace=True)


In [124]:
x_train

,impressions,clicks,cpc,cpr,cost,conversions
0,24,6,0.013333,0.250000,0.08,0
1,1,0,0.065381,0.000000,0.00,0
2,13,4,0.010000,0.307692,0.04,0
3,5,4,0.020000,0.800000,0.08,0
4,247,126,0.010238,0.510121,1.29,4
...,...,...,...,...,...,...
4566,19,6,0.011667,0.315789,0.07,0
4567,1,0,0.065381,0.000000,0.00,0
4568,16,8,0.015000,0.500000,0.12,1
4569,37,13,0.017692,0.351351,0.23,0


In [125]:
x_train = x_train.drop('cost',axis=1)
x_train = x_train.drop('clicks',axis=1)
x_train = x_train.drop('impressions',axis=1)
x_train

,cpc,cpr,conversions
0,0.013333,0.250000,0
1,0.065381,0.000000,0
2,0.010000,0.307692,0
3,0.020000,0.800000,0
4,0.010238,0.510121,4
...,...,...,...
4566,0.011667,0.315789,0
4567,0.065381,0.000000,0
4568,0.015000,0.500000,1
4569,0.017692,0.351351,0


In [126]:
x_test = x_test.drop('date',axis=1)
x_test = x_test.drop('campaign',axis=1)
x_test = x_test.drop('adgroup',axis=1)
x_test = x_test.drop('ad',axis=1)
x_test

,cost,impressions,clicks,conversions
0,0.58,121,49,1
1,0.17,22,12,0
2,0.05,5,3,0
3,0.01,2,1,0
4,0.01,3,1,0
...,...,...,...,...
313,0.01,1,1,0
314,0.01,5,1,0
315,0.14,25,11,0
316,0.06,9,4,0


In [127]:
cpr=x_test["clicks"]/x_test["impressions"]
cpc=x_test["cost"]/x_test["clicks"]

x_test.insert(2, "cpr", cpr)
x_test.insert(2, "cpc", cpc)
x_test['cpc'].fillna((x_test['cpc'].mean()), inplace=True)

x_test

,cost,impressions,cpc,cpr,clicks,conversions
0,0.58,121,0.011837,0.404959,49,1
1,0.17,22,0.014167,0.545455,12,0
2,0.05,5,0.016667,0.600000,3,0
3,0.01,2,0.010000,0.500000,1,0
4,0.01,3,0.010000,0.333333,1,0
...,...,...,...,...,...,...
313,0.01,1,0.010000,1.000000,1,0
314,0.01,5,0.010000,0.200000,1,0
315,0.14,25,0.012727,0.440000,11,0
316,0.06,9,0.015000,0.444444,4,0


In [128]:
x_test = x_test.drop('cost',axis=1)
x_test = x_test.drop('clicks',axis=1)
x_test = x_test.drop('impressions',axis=1)
x_test

,cpc,cpr,conversions
0,0.011837,0.404959,1
1,0.014167,0.545455,0
2,0.016667,0.600000,0
3,0.010000,0.500000,0
4,0.010000,0.333333,0
...,...,...,...
313,0.010000,1.000000,0
314,0.010000,0.200000,0
315,0.012727,0.440000,0
316,0.015000,0.444444,0


In [135]:
from sklearn.ensemble import RandomForestRegressor
LR = RandomForestRegressor(n_estimators=10, max_depth=None,min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None,bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False)
# fitting the training data
LR.fit(x_train,y_train)

RandomForestRegressor(n_estimators=10, n_jobs=1)

In [136]:
y_prediction =  LR.predict(x_test)
df=pd.DataFrame(y_prediction)

In [137]:
df.rename(columns = {0: 'revenue'}, inplace = True)

df

,revenue
0,129.706
1,0.000
2,0.000
3,0.000
4,0.000
...,...
313,0.000
314,0.000
315,0.000
316,0.000


In [84]:
df=df["revenue"]
df

0      161.819
1        0.000
2        0.000
3        0.000
4        0.000
        ...   
313      0.000
314      0.000
315      0.000
316      0.000
317    264.091
Name: revenue, Length: 318, dtype: float64

In [138]:
file=df.to_csv(r'Revenue.csv') 